In [37]:
from llama_index.core import SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, SummaryIndex
from llama_index.core.tools import QueryEngineTool
from llama_index.core.objects import ObjectIndex
from llama_index.core.agent import AgentRunner, FunctionCallingAgentWorker
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.ollama import Ollama
import os

DOCUMENT_PATHS = [
    r'C:\Users\ASUS\Downloads\Hamrah.txt',
    r'C:\Users\ASUS\Downloads\vape.txt',
    r'C:\Users\ASUS\Downloads\Shah.txt',
    r'C:\Users\ASUS\Downloads\Khalife.txt',
    r'C:\Users\ASUS\Downloads\carbon.txt',
    r'C:\Users\ASUS\Downloads\takapoo.txt'
]

EMBEDDING_MODEL = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
CHUNK_SIZE = 1000
OVERLAP = 200
LLM_MODEL = 'partai/dorna-llama3'

Settings.embed_model = HuggingFaceEmbedding(model_name=EMBEDDING_MODEL)
Settings.llm = Ollama(model=LLM_MODEL, temperature=0.1)
Settings.chunk_size = CHUNK_SIZE
Settings.chunk_overlap = OVERLAP
class PersianDocumentAgent:
    def __init__(self):
        self.splitter = SentenceSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=OVERLAP)
        self.tools = []  # Initialize tools as an empty list
        self.agent = None  # Initialize agent as None
        self.tools.extend()
        self.tools.append()
        self.initialized = False  # Initialize agent as None

    def _process_document(self, path: str, doc_id: int):
        """Process individual document and create tools"""
        documents = SimpleDirectoryReader(input_files=[path]).load_data()
        nodes = self.splitter.get_nodes_from_documents(documents)

        for node in nodes:
            node.metadata.update({
                "doc_id": doc_id,
                "doc_name": os.path.basename(path)
            })

        vector_index = VectorStoreIndex(nodes)
        summary_index = SummaryIndex(nodes)

        vector_engine = vector_index.as_query_engine(
            similarity_top_k=3,
            response_mode="compact"  # Better for local models
        )

        summary_engine = summary_index.as_query_engine(
            response_mode="tree_summarize",
            use_async=False  # Disable async for local model stability
        )

        doc_name = os.path.basename(path).replace('.txt', '')
        vector_tool = QueryEngineTool.from_defaults(
            query_engine=vector_engine,
            description=f"ابزار بازیابی اطلاعات از سند {doc_name} (شناسه: {doc_id}). "
                        "برای سوالات خاص درباره محتوای سند استفاده شود."
        )

        summary_tool = QueryEngineTool.from_defaults(
            query_engine=summary_engine,
            description=f"ابزار خلاصه‌سازی سند {doc_name} (شناسه: {doc_id}). "
                        "برای درخواست‌های کلی و خلاصه محتوا استفاده شود."
        )

        return [vector_tool, summary_tool]

    def initialize_agent(self):
        """Initialize agent with all document tools"""
        for doc_id, path in enumerate(DOCUMENT_PATHS):
            self.tools.extend(self._process_document(path, doc_id))  # Ensure tools are populated

        obj_index = ObjectIndex.from_objects(
            self.tools,
            index_cls=VectorStoreIndex,
        )
        obj_retriever = obj_index.as_retriever(similarity_top_k=2)  # Reduce for local model

        agent_worker = FunctionCallingAgentWorker.from_tools(
            tool_retriever=obj_retriever,
            llm=Settings.llm,
            system_prompt="""
            شما یک دستیار هوشمند فارسی هستید که باید به سوالات بر اساس اسناد ارائه شده پاسخ دهید.
            همیشه از ابزارهای موجود برای پاسخ استفاده کنید.
            پاسخ‌ها باید مختصر و دقیق باشد.
            اگر پاسخی پیدا نشد عبارت 'پاسخی یافت نشد' را برگردانید.
            """,
            verbose=True
        )
        self.agent = AgentRunner(agent_worker)

    def query_document(self, question: str, doc_id: int):
        """Execute a document-specific query"""
        augmented_query = f"{question} (مربوط به سند با شناسه {doc_id})"
        response = self.agent.query(augmented_query)
        return str(response).strip()

    doc_agent = PersianDocumentAgent()
    doc_agent.initialize_agent()

    queries = [
        ("چرا اینترنت همراه اول گوشی وصل نمیشود؟", 0),
        ("چطوری ویپ مورد نظرمو پیدا کنم؟", 1),
        ("شاه عباس که بود؟", 2),
        ("خلیفه سلطان که بود و چه کرد؟", 3),
        ("کربن اکتیو و کربن بلک چه هستند و چه تفاوتی دارند و برای چه استفاده میشن؟", 4),
        ("شرکت تکاپو صنعت نامی چه محصولاتی ارایه میدهد؟ چه چیزی این شرکت را منحصر به فرد میسازد؟ سهام این شرکت صعودی است یا نزولی؟", 5)
    ]

    with open(r'C:\Users\ASUS\Downloads\representation3.txt', 'w', encoding='utf-8') as f:
        for q_idx, (query, doc_idx) in enumerate(queries):
            answer = doc_agent.query_document(query, doc_idx)

            f.write(f"سوال {q_idx+1} ({doc_idx+1}):\n{query}\n\nپاسخ:\n{answer}\n\n{'='*50}\n\n")
            print(f"پردازش سوال {q_idx+1}/{len(queries)} تکمیل شد")

    print("تمامی سوالات با موفقیت پردازش شدند!")

AttributeError: 'PersianDocumentAgent' object has no attribute 'tools'

In [35]:
agent = PersianDocumentAgent()
print(dir(agent))  # لیست تمام ویژگی‌ها و متدهای این آبجکت را نمایش می‌دهد
# pip install --upgrade <PersianDocumentAgent>

# tools = getattr(agent, 'tools', None)
# if tools is None:
#     print("ویژگی tools وجود ندارد!")
# else:
#     print("ویژگی tools مقدار دارد:", tools)
#
# doc_agent = PersianDocumentAgent()


['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_process_document', 'init', 'initialize_agent', 'query_document']


In [13]:
if name == "main":
    doc_agent = PersianDocumentAgent()
    doc_agent.initialize_agent()

    queries = [
        ("چرا اینترنت همراه اول گوشی وصل نمیشود؟", 0),
        ("چطوری ویپ مورد نظرمو پیدا کنم؟", 1),
        ("شاه عباس که بود؟", 2),
        ("خلیفه سلطان که بود و چه کرد؟", 3),
        ("کربن اکتیو و کربن بلک چه هستند و چه تفاوتی دارند و برای چه استفاده میشن؟", 4),
        ("شرکت تکاپو صنعت نامی چه محصولاتی ارایه میدهد؟ چه چیزی این شرکت را منحصر به فرد میسازد؟ سهام این شرکت صعودی است یا نزولی؟", 5)
    ]

    with open(r'C:\Users\User\Downloads\Telegram Desktop\data_llamaindex.txt', 'w', encoding='utf-8') as f:
        for q_idx, (query, doc_idx) in enumerate(queries):
            answer = doc_agent.query_document(query, doc_idx)

            f.write(f"سوال {q_idx+1} ({doc_idx+1}):\n{query}\n\nپاسخ:\n{answer}\n\n{'='*50}\n\n")
            print(f"پردازش سوال {q_idx+1}/{len(queries)} تکمیل شد")

print("تمامی سوالات با موفقیت پردازش شدند!")

NameError: name 'name' is not defined

In [28]:
class MyClass:
    def __init__(self):
        self.tools = []  # Initialize as an empty list

    def add_tool(self, tool):
        """Add a tool to the tools list."""
        self.tools.append(tool)

    def show_tools(self):
        """Print all tools."""
        print("Tools:", self.tools)

# Example Usage
obj = MyClass()
obj.add_tool("Hammer")
obj.add_tool("Screwdriver")
obj.show_tools()


Tools: ['Hammer', 'Screwdriver']


In [30]:

agent = MyClass()
print(dir(agent))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_tool', 'show_tools', 'tools']
